In [12]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

torch.__version__

'0.3.1.post2'

In [17]:
batch_size = 64
momentum = 0.5
test_batch_size = 1000
lr = 0.01
log_interval = 10
epochs = 10

use_cuda = torch.cuda.is_available()
seed = 3123
torch.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed(seed)

In [18]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()
if use_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298004
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.289057
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.285439
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.272910
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.262873
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.223264
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.217230
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.174353
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.137086
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.086521
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.070718
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.966059
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.872754
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.777403
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.670768
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.553841
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.536084
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.585584
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.273776
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.183107
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.366170
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.485396
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.387595
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.336641
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.295289
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.252143
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.209793
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.375131
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.197359
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.496797
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.530404
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.144795
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.290489
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.255976
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.178437
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.330529
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.218725
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.214931
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.472209
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.403778


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.284243
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.188774
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.180043
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.240521
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.270374
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.281689
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.185480
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.266213
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.435247
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.249161
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.213103
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.257162
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.159978
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.087141
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.191304
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.164352
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.288206
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.367796
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.603637
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.161895


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.088696
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.161623
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.222889
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.188256
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.211225
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.191737
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.255439
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.244145
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.127003
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.090442
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.146439
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.129516
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.172348
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.208239
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.253611
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.148933
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.131173
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.371561
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.164661
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.123439


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.053163
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.115157
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.131712
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.388450

Test set: Average loss: 0.0609, Accuracy: 9796/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.125954
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.191458
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.081414
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.388731
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.251465
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.306294
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.262256
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.149468
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.102164
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.215381
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.390650
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.134472
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.086967
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.211009
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.164715
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.147481
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.233274
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.077488
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.140515
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.242941
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.118011
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.270971
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.197301
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.218152
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.218343
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.097360
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.258072
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.224643
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.091834
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.109879
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.181845
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.155836
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.166946
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.175933
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.066944
